In [1]:
from networkit import *

All considered sparsification algorithm implementations rely on edge attributes, so do not forget to call indexEdges() on the graph you want to work on.

In [2]:
G = readGraph("/home/gerd/workspace/Papers/Sparsification/scripts/input/Caltech36.graphml", Format.GraphML)
G.indexEdges()
properties.size(G)

(769, 16656)

Sparsification algorithms that leave the set of nodes intact can be split up into edge attribute calculation and a global filtering step. All backbone algorithm implementations in the backbones-module are based on that insight. The module provides both low-level  attributizers and filters and high-level convenience classes which can be identified by the suffix 'Backbone'.

### Example 1: Simple calculation of a sparsified graph

In [40]:
sparsificationAlgorithm = sparsification.LocalSimilarityBackbone()

In the following code piece, we need to specify a parameter value, which indirectly influences the size of the resulting graph.

In [41]:
S = sparsificationAlgorithm.getSparsifiedGraph(G, 0.5)
properties.size(S)

(769, 3194)

In order to obtain a reduced graph of a specific size, we can use the following convenience function which applies an appropriate parameterization algorithm to obtain a parameter value first:

In [42]:
S = sparsificationAlgorithm.getSparsifiedGraphOfSize(G, 0.5)
properties.size(S)

(769, 8328)

Note that pre-calculated edge attributes can be passed to these functions as well in order to avoid unnecessary recalculations.

### Example 2: Exporting a sparsification attribute to Gephi

In order to filter edges within Gephi, we need to calculate the edge attribute:

In [8]:
sparsificationAlgorithm = sparsification.LocalDegreeBackbone()
edgeAttribute = sparsificationAlgorithm.getAttribute(G)

We can now export it to Gephi (start the Gephi Streaming Master server first):

In [4]:
gephiClient = gephi.streaming.GephiStreamingClient()
gephiClient.exportGraph(G)
gephiClient.exportEdgeValues(G, edgeAttribute, 'myAttribute')

### Example 3: Listing of all implemented sparsification definitions

Note that it is important to consider, how the respective edge attributes are to be interpreted (are those edges with attribute values below or above a certain threshold to be kept in the graph?). A convention to reduce confusion might be introduced here in the future.

In [3]:
sAlgorithms = [
    (sparsification.ForestFireBackbone(0.15, 5), 'Forest Fire'),
    (sparsification.LocalDegreeBackbone(), 'Local Degree'),
    (sparsification.LocalSimilarityBackbone(), 'Local Similarity'),
    (sparsification.MultiscaleBackbone(), 'Multiscale'),
    (sparsification.RandomEdgeBackbone(), 'RandomEdge'),
    (sparsification.RandomNodeEdgeBackbone(), 'RandomNodeEdge'),
    (sparsification.SimmelianBackboneNonParametric(), 'Simmelian NonParametric'),
    (sparsification.SimmelianBackboneParametric(10), 'Simmelian Parametric'),
    (sparsification.SimmelianQuadrangleBackbone(), 'Simmelian Quadrangle'),
    (sparsification.SCANBackbone(), 'SCAN'),
]

Let's calculate an edge attribute for each of these definitions and export them all to Gephi:

In [4]:
gephiClient = gephi.streaming.GephiStreamingClient()
gephiClient.clearGraph()
gephiClient.exportGraph(G)
for (algorithm, name) in sAlgorithms:
    edgeAttribute = algorithm.getAttribute(G)
    rankAttribute = sparsification.getRankAttribute(edgeAttribute)
    gephiClient.exportEdgeValues(G, rankAttribute, name)

For visualization purposes, let's also calculate and export the community structure:

In [5]:
c = community.detectCommunities(G, community.PLM(G, refine=False, par='none'))
gephiClient.exportNodeValues(G, c, 'community')

PLM(none,pc) detected communities in 0.07216715812683105 [s]
solution properties:
-------------------  ----------
# communities         11
min community size     2
max community size   159
avg. community size   69.9091
modularity             0.398057
-------------------  ----------


### Example 4: Using attributes, filters, and parameterization

In the following example, we illustrate possible usage of an attributizer, a filter and a parameterization algorithm. Note that any of these might be exchanged.

In [48]:
#Attribute calculation
sparsificationAlgorithm = sparsification.LocalDegreeBackbone()
attribute = sparsificationAlgorithm.getAttribute(G)

#Parameterization using binary search with up to 20 steps. The parameter value is in [0,1] and the size of the graph increases with increasing parameter value.
parameterization = sparsification.BinarySearchParameterization(True, 0.0, 1.0, 20)
parameter = parameterization.parameterize(sparsificationAlgorithm, G, attribute, 0.3) # We'd like to keep ~30% of edges

#Global filtering
globalFilter = sparsification.GlobalThresholdFilter(G, attribute, parameter, False) #Keep all edges with an edge attribute value below the given parameter
B = globalFilter.calculate()
properties.size(B)

(769, 4998)

### Example 5: Using rank attributes with Gephi

An example where it might be desireable to work with 'linear' edge attributes is Filtering in Gephi. By 'linear' we mean that a value close to the mean value should yield a graph with about 50% of edges. The sparsification module offers a convenience function that offers this functionality and that can be applied to both node and edge attributes.

In [49]:
sparsificationAlgorithm = sparsification.LocalSimilarityBackbone()
attribute = sparsificationAlgorithm.getAttribute(G)
rankAttribute = sparsification.getRankAttribute(attribute)
gephiClient.exportEdgeValues(G, rankAttribute, 'rank')